# Programming Assignment 1 - A statevector simulator

(This exercise is based on a [QOSF](https://qosf.org/) [Task](https://github.com/quantastica/qosf-mentorship/blob/master/qosf-simulator-task.ipynb))

The goal is to implement a simple quantum circuit simulator. In general, a quantum circuit simulator should be capable of performing the following:

- initialize state (see [DiVincenzo criteria](https://en.wikipedia.org/wiki/DiVincenzo%27s_criteria))
- read program
- calculate matrix operator
- apply operator (modify state)
- perform multi-shot measurement of all qubits using weighted random technique


However, in order to implement simulator, it is best if you have function which returns operator for any unitary targeting any qubit(s).
- [qosf-simulator-task-additional-info.pdf](https://github.com/quantastica/qosf-mentorship/blob/master/qosf-simulator-task-additional-info.pdf)
- Book *Nielsen, Michael A.; Chuang, Isaac (2000). Quantum Computation and Quantum Information, 10th Anniversary Edition, Section 8.2.3, Operator-Sum Representation*

## Background

### Qubit

Qubit is the basic unit of quantum information. It is a two-state (or two-level) quantum-mechanical system, and can be represented by a linear superposition of its two orthonormal basis states (or basis vectors). The vector representation of a single qubit is: ${\vert a\rangle =v_{0} \vert 0\rangle +v_{1} \vert 1\rangle \rightarrow {\begin{bmatrix}v_{0}\\v_{1}\end{bmatrix}}}$,
Here, ${\displaystyle v_{0}}v_{0}$ and ${\displaystyle v_{1}}v_{1}$ are the complex probability amplitudes of the qubit. These values determine the probability of measuring a 0 or a 1, when measuring the state of the qubit.

Code:

In [3]:
# Qubit in |0> state (100% probability of measuring 0)
# Remember this line for the exercise, when implementing initialisation
q0 = [1, 0]

# Qubit in |1> state (100% probability of measuring 1)
q1 = [0, 1] 

# Qubit |+> state (superposition: 50% probability of measuring 0 and 50% probability of measuring 1)
q2 = [0.7071067811865475, 0.7071067811865475]

# Qubit |-> state (superposition: 50% probability of measuring 0 and 50% probability of measuring 1) with phase pi
q3 = [0.7071067811865475, -0.7071067811865475]

# Qubit |i> state (superposition: 50% probability of measuring 0 and 50% probability of measuring 1) with phase pi/2
q3 = [0.7071067811865475, 0+0.7071067811865475j]

# Qubit |-i> state (superposition: 50% probability of measuring 0 and 50% probability of measuring 1) with phase -pi/2
q4 = [0.7071067811865475, 0-0.7071067811865475j]


Note that vector contains probability amplitudes - not probabilities. Probability amplitude is complex number and can be negative. Probability is calculated as absolute value squared:

In [4]:
import numpy as np

# We use complex numbers, e.g. 0.7071067811865475+0j
q4 = np.array([0.7071067811865475+0j, 0-0.7071067811865475j])

# The probability is the square of the absolute values
# Remember this line for the exercise, when implementing measurements
p4 = np.abs(q4) ** 2
print(p4)

[0.5 0.5]


#### State vector

The combined state of multiple qubits is the tensor product of their states (vectors). The [tensor product](https://en.wikipedia.org/wiki/Tensor_product) is denoted by the symbol ${\displaystyle \otimes }$.

The vector representation of two qubits is:

${\displaystyle \vert ab\rangle =\vert a\rangle \otimes \vert b\rangle =v_{00}\vert 00\rangle +v_{01}\vert 01\rangle +v_{10}\vert 10\rangle +v_{11}\vert 11\rangle \rightarrow {\begin{bmatrix}v_{00}\\v_{01}\\v_{10}\\v_{11}\end{bmatrix}}}$

Example:

In [5]:
# Qubit in |0> state (100% probability of measuring 0)
q0 = [1, 0]

# Qubit in |1> state (100% probability of measuring 1)
q1 = [0, 1] 

# https://numpy.org/doc/stable/reference/generated/numpy.kron.html#numpy.kron
combined_state = np.kron(q0, q1)

# [1*0, 1*1, 0*0, 0*1]
print(combined_state)

[0 1 0 0]


Now, what this vector tells us?

It will be more clear if we write vector elements in a column with index expressed in binary format:

```
Index (dec)  Index (bin)  Amplitude  Probability
================================================
0            00           0          0 (  0%)
1            01           1          1 (100%)
2            10           0          0 (  0%)
3            11           0          0 (  0%)
```

- First element (binary: 00) is probability of measuring 0 on both qubits.
- Second element (binary: 01) is probability of measuring 0 on first qubit and 1 on second qubit.
- Third element (binary: 10) is probability of measuring 1 on first qubit and 0 on second qubit.
- Fourth element (binary: 11) is probability of measuring 1 on both qubits.

#### Endianness

It is important to say that different quantum programming frameworks use different orientation of bitstrings ([endianness](https://en.wikipedia.org/wiki/Endianness)). In previous example, left bit belongs to first qubit and right bit belongs to second qubit. This enconding is called "big endian".

But, in some frameworks (like Qiskit), encoding is opposite: rightmost bit belongs to first qubit and leftmost bit belongs to last qubit. This is called "little endian".

So, vector from our example in Qiskit's "little endian" encoding will look like this:

```
Index (dec)  Index (bin)  Amplitude  Probability
================================================
0            00           0          0 (  0%)
1            01           0          0 (  0%)
2            10           1          1 (100%)
3            11           0          0 (  0%)
```

"Little endian" encoding:

- First element (binary: 00) is probability of measuring 0 on both qubits.
- Second element (binary: 01) is probability of measuring 0 on second qubit and 1 on first qubit.
- Third element (binary: 10) is probability of measuring 1 on second qubit and 0 on first qubit.
- Fourth element (binary: 11) is probability of measuring 1 on both qubits.


### Quantum gates

Quantum gates are basic units of quantum processing. Gates are represented as [unitary matrices](https://en.wikipedia.org/wiki/Unitary_matrix) (e.g. have determinant one).

Quantum gates (usually) act on small number of qubits. We have single-qubit and multi-qubit gates. 

The action of the gate on a specific quantum state is found by multiplying the vector ${\displaystyle \vert \psi _{1}\rangle }$  which represents the state, by the matrix ${\displaystyle U}$ representing the gate. The result is a new quantum state ${\displaystyle \vert \psi _{2}\rangle }$ 

${\displaystyle U\vert \psi _{1}\rangle =\vert \psi _{2}\rangle }$


**Exponential size**: n-qubit gate is represented as $2^n\times2^n$ unitary matrix.

Examples:

#### Single qubit gates

X (aka NOT, bit flip) gate is part of a more general set of gates known as [Pauli gates](https://en.wikipedia.org/wiki/Quantum_logic_gate#Pauli-X_gate):

$X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$

Hadamard gate:

$H = \tfrac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}$

T gate:

$T = \begin{bmatrix} 1 & 0 \\ 0 & e^{i\frac{\pi}{4}} \end{bmatrix}$

General single qubit rotation gate:

$U_3(\theta, \phi, \lambda) = 
\begin{bmatrix} \cos(\theta/2) & -e^{i\lambda}\sin(\theta/2) \\ 
                e^{i\phi}\sin(\theta/2) & e^{i\lambda+i\phi}\cos(\theta/2)
\end{bmatrix}$

#### Two-qubit gates:

${CNOT} =
\begin{bmatrix}
      1 & 0 & 0 & 0 \\
      0 & 1 & 0 & 0 \\
      0 & 0 & 0 & 1 \\
      0 & 0 & 1 & 0 \\
\end{bmatrix}$

${SWAP} = 
\begin{bmatrix}
    1 & 0 & 0 & 0 \\
    0 & 0 & 1 & 0 \\
    0 & 1 & 0 & 0 \\
    0 & 0 & 0 & 1 \\
\end{bmatrix}$

#### Examples

Let's see how single-qubit gate modifies state of the qubit:

In [6]:
import numpy as np

# Let's start with qubit in state |0> (100% probability of measuring 0)   
q0 = np.array([1, 0])

print("Initial state:\t", q0)

# Define X (NOT) gate:
X = np.array([
  [0, 1],
  [1, 0]
])

# Now apply X gate to a qubit (matrix-vector dot product):
q0 = np.dot(X, q0)
print("1. Final state:\t", q0)

q0 = np.dot(X, q0)
print("2. Final state:\t", q0)

Initial state:	 [1 0]
1. Final state:	 [0 1]
2. Final state:	 [1 0]


After applying X gate, qubit flips from state $|0\rangle$ to state $|1\rangle$.

Now, let's see how Hadamard gate works:

In [7]:
import numpy as np

# Let's start with qubit in state |0> (100% probability of measuring 0)
q0 = np.array([1, 0])
print("Initial state:\t", q0)

# Define H (Hadamard) gate:
H = np.array([
  [1/np.sqrt(2), 1/np.sqrt(2)],
  [1/np.sqrt(2), -1/np.sqrt(2)]
])

# Now apply H gate to a qubit (matrix-vector dot product):
# q0 = np.dot(X, q0)
q0 = np.dot(H, q0)
print("Final state:\t", q0)

Initial state:	 [1 0]
Final state:	 [0.70710678 0.70710678]


After applying Hadamard gate on qubit in state $|0\rangle$ it evolves to state $|+\rangle$ which is equal superposition.

### Matrix operator

Quantum program evolves quantum state by multiplying state vector with each gate's unitary matrix (dot product). Note that dimension of the state vector and dimension of the unitary matrix describing a gate usually don't match. For example: 3-qubit quantum circuit's state vector has $2^n=2^3=8$ elements, but single-qubit gate has $2^n\times2^n=2^1\times2^1=2\times2$ elements. In order to perform matrix-vector multiplication, we need to "resize" gate's matrix to the dimension of the state vector. Let's call that matrix a **matrix operator**.

Note that size of the matrix operator is $2^n\times2^n$ where $n$ is total number of qubits in the circuit, so storing it into memory and calculating it requires a lot of memory and cpu power for bigger circuits. Optimizing this code is most interesting and challenging part, but for our purpose it is enough if you make it work smoothly with 8 qubits (the more - the better).

#### Matrix operator for single-qubit gates

Matrix operator for single-qubit gate can be calculated by performing tensor product of gate's unitary matrix and $2\times2$ identity matrices in correct order.

Example for single-qubit gate $U$ in 3-qubit circuit:

- gate on qubit 0: ${O=U \otimes I \otimes I}$
- gate on qubit 1: ${O=I \otimes U \otimes I}$
- gate on qubit 2: ${O=I \otimes I \otimes U}$

Example matrix operator for X gate acting on third qubit in 3-qubit circuit can be calculated like this:

In [8]:
import numpy as np

# Let's define state vector of the 3-qubit circuit in "ground state" (all qubits in state |0>)
# |000>
psi = [1, 0, 0, 0, 0, 0, 0, 0]
print("Initial state:", psi)

# Define X (NOT) gate:
X = np.array([
  [0, 1],
  [1, 0]
])

# Define 2x2 identity
I = np.identity(2)

# The maximum number of qubits in the circuit
# The state psi has three qubits,  too
NR_MAX_Q = 3

# Calculate operator for X gate acting on third qubit in 3-qubit circuit
# I I X
O = 1

index_single_qubit_gate = 2

for i in range(NR_MAX_Q):
  if i == index_single_qubit_gate:
    O = np.kron(O, X)
  else:
    O = np.kron(O, I)

print("\nOperator:\n", O, "\n")

# And finally, apply operator
# |001>
psi = np.dot(psi, O)
print("Final state:", psi)

Initial state: [1, 0, 0, 0, 0, 0, 0, 0]

Operator:
 [[0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]] 

Final state: [0. 1. 0. 0. 0. 0. 0. 0.]


We are dealing with "big endian" encoding, so this result is correct: third qubit is flipped to state $|1\rangle$  and other qubits are not changed.

**Note**: if we want vector in "little endian" encoding (like Qiskit), then order in which we perform tensor product to calculate operator is opposite. Instead ${O=I \otimes I \otimes U}$ we would do ${O=U \otimes I \otimes I}$.

#### Matrix operator for multi-qubit gates

If we want to apply two qubit gate on subsequent qubits ( 0-1, 1-2, 2-3 etc.) then we can use the same technique like we do with single qubit gates:

For 3-qubit circuit, CNOT gate (see code below):

- acting on first and second qubit, operator is ${O=CNOT \otimes I}$

- acting on second and third qubit, operator is ${O=I \otimes CNOT}$

But, multi-qubit gates can be applied to qubits which are not consequent, so this is not that trivial.

The main feature of a controlled-$U$ operation, for any unitary $U$, is that it (coherently) performs an operation on some qubits depending on the value of some single qubit. The way that we can write this explicitly algebraically (with the control on the first qubit) is:

$\mathit{CU} \;=\; \vert{0}\rangle\!\langle{0}\vert \!\otimes\! \mathbf 1 \,+\, \vert{1}\rangle\!\langle{1}\vert \!\otimes\! U$

where ${\mathbf 1}$ is an identity matrix of the same dimension as $U$.

Here, ${\vert{0}\rangle\!\langle{0}\vert}$ and ${\vert{1}\rangle\!\langle{1}\vert}$ are projectors onto the states ${\vert{0}\rangle}$ and ${\vert{1}\rangle}$ of the control qubit &mdash; but we are not using them here as elements of a measurement, but to describe the effect on the other qubits depending on one or the other subspace of the state-space of the first qubit.

We can use this to derive the matrix for the gate ${\mathit{CX}_{1,3}}$ which performs an $X$ operation on qubit 3, coherently conditioned on the state of qubit 1, by thinking of this as a controlled-${(\mathbf 1_2 \!\otimes\! X)}$ operation on qubits 1 and 3:

$\begin{aligned}
\mathit{CX}_{1,3} \;&=\;
\vert{0}\rangle\!\langle{0}\vert \otimes \mathbf 1_4 \,+\, \vert{1}\rangle\!\langle{1}\vert \otimes (\mathbf 1_2 \otimes X)
\\[1ex]&=\;
\begin{bmatrix}
  \mathbf 1_4 & \mathbf 0_4  \\
  \mathbf 0_4 & (\mathbf 1_2 \!\otimes\! X) 
\end{bmatrix}
\;=\;
\begin{bmatrix}
  \mathbf 1_2 & \mathbf 0_2 & \mathbf 0_2 & \mathbf 0_2 \\
  \mathbf 0_2 & \mathbf 1_2 & \mathbf 0_2 & \mathbf 0_2 \\
  \mathbf 0_2 & \mathbf 0_2 & X & \mathbf 0_2 \\
  \mathbf 0_2 & \mathbf 0_2 & \mathbf 0_2 & X
\end{bmatrix},
\end{aligned}$

where the latter two are block matrix representations to save on space (and sanity).

Better still: we can recognise that &mdash; on some mathematical level where we allow ourselves to realise that the order of the tensor factors doesn't have to be in some fixed order &mdash; the control and the target of the operation can be on any two tensor factors, and that we can fill in the description of the operator on all of the other qubits with $\mathbf 1_2$. This would allow us to jump straight to the representation

$\begin{aligned}
\mathit{CX}_{1,3} \;&=&\;
\underbrace{\vert{0}\rangle\!\langle{0}\vert}_{\text{control}} \otimes \underbrace{\;\mathbf 1_2\;}_{\!\!\!\!\text{uninvolved}\!\!\!\!} \otimes \underbrace{\;\mathbf 1_2\;}_{\!\!\!\!\text{target}\!\!\!\!}
&+\,
\underbrace{\vert{1}\rangle\!\langle{1}\vert}_{\text{control}} \otimes \underbrace{\;\mathbf 1_2\;}_{\!\!\!\!\text{uninvolved}\!\!\!\!} \otimes \underbrace{\; X\;}_{\!\!\!\!\text{target}\!\!\!\!}
\\[1ex]&=&\;
\begin{bmatrix}
  \mathbf 1_2 & \mathbf 0_2 & \phantom{\mathbf 0_2} & \phantom{\mathbf 0_2} \\
  \mathbf 0_2 & \mathbf 1_2 & \phantom{\mathbf 0_2} & \phantom{\mathbf 0_2} \\
  \phantom{\mathbf 0_2} & \phantom{\mathbf 0_2} & \phantom{\mathbf 0_2} & \phantom{\mathbf 0_2} \\
  \phantom{\mathbf 0_2} & \phantom{\mathbf 0_2} & \phantom{\mathbf 0_2} & \phantom{\mathbf 0_2}
\end{bmatrix}
\,&+\,
\begin{bmatrix}
  \phantom{\mathbf 0_2} & \phantom{\mathbf 0_2} & \phantom{\mathbf 0_2} & \phantom{\mathbf 0_2} \\
  \phantom{\mathbf 0_2} & \phantom{\mathbf 0_2} & \phantom{\mathbf 0_2} & \phantom{\mathbf 0_2} \\
  \phantom{\mathbf 0_2} & \phantom{\mathbf 0_2} & X & \mathbf 0_2 \\
  \phantom{\mathbf 0_2} & \phantom{\mathbf 0_2} & {\mathbf 0_2} & X
\end{bmatrix}
\end{aligned}$

and also allows us to immediately see what to do if the roles of control and target are reversed:

$\begin{aligned}
\mathit{CX}_{3,1} \;&=&\;
\underbrace{\;\mathbf 1_2\;}_{\!\!\!\!\text{target}\!\!\!\!} \otimes \underbrace{\;\mathbf 1_2\;}_{\!\!\!\!\text{uninvolved}\!\!\!\!} \otimes \underbrace{\vert{0}\rangle\!\langle{0}\vert}_{\text{control}}
\,&+\,
\underbrace{\;X\;}_{\!\!\!\!\text{target}\!\!\!\!} \otimes \underbrace{\;\mathbf 1_2\;}_{\!\!\!\!\text{uninvolved}\!\!\!\!} \otimes \underbrace{\vert{1}\rangle\!\langle{1}\vert}_{\text{control}}
\\[1ex]&=&\;
{\scriptstyle\begin{bmatrix}
 \!\vert{0}\rangle\!\langle{0}\vert\!\! & & & \\
 & \!\!\vert{0}\rangle\!\langle{0}\vert\!\! & & \\
& & \!\!\vert{0}\rangle\!\langle{0}\vert\!\! & \\
& & & \!\!\vert{0}\rangle\!\langle{0}\vert
\end{bmatrix}}
\,&+\,
{\scriptstyle\begin{bmatrix}
 & & \!\!\vert{1}\rangle\!\langle{1}\vert\!\! & \\
 & & & \!\!\vert{1}\rangle\!\langle{1}\vert \\
\!\vert{1}\rangle\!\langle{1}\vert\!\! & & &  \\
& \!\!\vert{1}\rangle\!\langle{1}\vert & &
\end{bmatrix}}
\\[1ex]&=&\;
\left[{\scriptstyle\begin{matrix}
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1
\end{matrix}}\right.\,\,&\,\,\left.{\scriptstyle\begin{matrix}
0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0
\end{matrix}}\right].
\end{aligned}$

But best of all: if you can write down these operators algebraically, you can take the first steps towards dispensing with the giant matrices entirely, instead  reasoning about these operators algebraically using expressions such as $\mathit{CX}_{1,3} =
\vert{0}\rangle\!\langle{0}\vert \! \otimes\!\mathbf 1_2\! \otimes\! \mathbf 1_2 +
 \vert{1}\rangle\!\langle{1}\vert \! \otimes\!  \mathbf 1_2 \! \otimes\!  X$
and
$\mathit{CX}_{3,1} =
\mathbf 1_2 \! \otimes\! \mathbf 1_2 \! \otimes \! \vert{0}\rangle\!\langle{0}\vert +
X \! \otimes\! \mathbf 1_2 \! \otimes \! \vert{1}\rangle\!\langle{1}\vert$.

### Measurement

State vector of the real quantum computer cannot be directly observed. All we can read out of qubit is a single classical bit. So best we can get as output from quantum computer is bitstring of size $n$ where $n$ is number of qubits. Reading the state from a qubit is called "measurement". When qubit is in superposition, measurement puts qubit in one of two classical states. If we read 1 from qubit, it will "collapse" to state |1> and will stay there - superposition is "destroyed", and any subsequent measurement will return the same result.

Measurement is non-unitary operation on the state vector. But for simplicity, and because we can access state vector of our simulator, it is easier if we do it with a trick:

We can simulate measurement by choosing element from the state vector with weighted random function. Elements with larger probability amplitude will be returned more often, and elements with smaller probability amplitude will be returned less often. Elements with zero probability will never be returned.

For example, this state vector:

```
Index (dec)  Index (bin)  Amplitude   Probability
=================================================
0            00           0.70710678  0.5 (50%)
1            01           0           0   ( 0%)
2            10           0           0   ( 0%)
3            11           0.70710678  0.5 (50%)
```

Our random function should return elements 00 and 11 equaly often and never return 01 and 10. If we execute it 1000 times (1000 shots) we should get something like this:

```
{
  "00": 494,
  "11": 506
}
```
*(this is random, so it usually is not exactly 500/500 and that is completelly fine)*

## Declarations


## Input format (quantum program)
You can define unitaries in the code, and accept program with gate names:
```
[
  { "gate": "h", "qubits": [0] }, 
  { "gate": "cx", "qubits": [0, 1] }
  ...
]
```

**In the following, focus only on CNOT, H, and T gates. These gate form an universal gate set.**

In [9]:
import numpy as np
import math
import random
import collections

# Define X gate (CNOT is controlled-X):
X = np.array([
  [0, 1],
  [1, 0]
])

# Define 2x2 Identity
I = np.identity(2)

# Define Hadamard gate:
H = 1/np.sqrt(2)*np.array([
  [1, 1],
  [1, -1]
])

# Define T gate:
T = np.array([
  [1, 0],
  [0, np.exp(1j*np.pi/4)]
])

# Define projection operator |0><0|
P0x0 = np.array([
  [1, 0],
  [0, 0]
])

# Define projection operator |1><1|
P1x1 = np.array([
  [0, 0],
  [0, 1]
])

## Simulator Framework

In [10]:
import itertools

def get_ground_state(num_qubits):
    st = np.zeros(2 ** num_qubits)
    st[0] = 1
    return st

def get_operator(num_qubits, gate_unitary, qubits):
    op = 1
    op2 = 1
    if len(qubits) > 1:
        for i in range(num_qubits):
            if i == qubits[0]:
                op = np.kron(op, P0x0)
                op2 = np.kron(op2,P1x1)
            elif i == qubits[1]:
                op = np.kron(op, I)
                op2 = np.kron(op2, X)
            else:
                op = np.kron(op, I)  
                op2 = np.kron(op2, I)
        op = op + op2
    else:
        for i in range(num_qubits):
            if i == qubits[0]:
                op = np.kron(op, gate_unitary)
            else:
                op = np.kron(op, I)  

    return op

def run_program(program):
    # read program, and for each gate:
    #   - calculate matrix operator
    #   - multiply state with operator
    # return final state

    # Find out num qubits
    num_qubits = 0
    for op in program:
        if num_qubits < max(op["qubits"]):
            num_qubits = max(op["qubits"])
    # Indices run from zero, but number starts from 1
    num_qubits += 1
    print("maximum number of qubits", num_qubits)

    # See diVincenzo criteria -- init to |00000...0>
    # In this example, "|0>"
    state = get_ground_state(num_qubits)
    # print("input:", state)

    output_state = np.array(state)
    un = 0
    for operation in program:
        if operation["gate"] == "h":
            # print("we do Hadamard!")
            un = get_operator(num_qubits, H, operation["qubits"])
        if operation["gate"] == "cx":
            # print("we do CNOT!")
            un = get_operator(num_qubits, X, operation["qubits"])
        if operation["gate"] == "t":
            # print("we do T!")
            un = get_operator(num_qubits, T, operation["qubits"])
    
        output_state = np.dot(un, output_state)

    return output_state


def measure_all(state_vector):
    res = np.square(np.absolute(state_vector))

    num_qubits = int(np.log2(len(state_vector)))
    outcomes = ["".join(map(str, x)) for x in itertools.product([0, 1], repeat=num_qubits)]

    return random.choices(outcomes, weights=res, k=1)[0]

def get_counts(state_vector, num_shots):  
    lista = []
    for i in range(num_shots):
        state_str = ( f"{measure_all(state_vector)}")
        lista.append(state_str)

    counter = collections.Counter(lista)
    print(counter)
    return counter

## Test Circuits

In [11]:
ghz_with_zero = [
  { "gate": "h", "qubits": [1] },
  { "gate": "cx", "qubits": [1, 0] },
  { "gate": "cx", "qubits": [1, 3] },
  { "gate": "cx", "qubits": [1, 4] },
]

swap01 = [
  { "gate": "h", "qubits": [0] },
  { "gate": "t", "qubits": [0] },
  { "gate": "t", "qubits": [0] },
  { "gate": "t", "qubits": [0] },
  { "gate": "t", "qubits": [0] },
  { "gate": "h", "qubits": [0] },
  { "gate": "cx", "qubits": [1, 0] },
  { "gate": "cx", "qubits": [0, 1] },
  { "gate": "cx", "qubits": [1, 0] },
]

# For the above test circuits the outputs should look like the ones below
# 
#  *** GHZ
# maximum number of qubits 5
# Counter({'00000': 522, '11011': 478})
#
#
#  *** Swap 01
# maximum number of qubits 2
# Counter({'01': 1000})
#

print("\n\n *** GHZ")
final_state = run_program(ghz_with_zero)
counts = get_counts(final_state, 1000)

assert (len(counts.keys()) == 2)
assert '00000' in counts.keys()
assert '11011' in counts.keys()
assert counts['00000'] + counts['11011'] == 1000
assert 450 <= counts['00000'] <= 550
assert 450 <= counts['11011'] <= 550
print(counts)

print("\n\n *** Swap 01")
final_state = run_program(swap01)
counts = get_counts(final_state, 1000)
assert len(counts.keys()) == 1
assert '01' in counts.keys()
assert counts['01'] == 1000
print(counts)



 *** GHZ
maximum number of qubits 5
Counter({'11011': 543, '00000': 457})
Counter({'11011': 543, '00000': 457})


 *** Swap 01
maximum number of qubits 2
Counter({'01': 1000})
Counter({'01': 1000})
